In [1]:
from ragas.testset import TestsetGenerator
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from ragas.llms import LangchainLLM

# Create Test set

In [2]:
loader = PyPDFLoader("../data/Batman_wiki.pdf")
pages = loader.load()

In [3]:
# Add custom llms and embeddings
generator_llm = LangchainLLM(ChatOpenAI(model="gpt-3.5-turbo"))
critic_llm = LangchainLLM(ChatOpenAI(model="gpt-4"))
embeddings_model = OpenAIEmbeddings()

# Change resulting question type distribution
testset_distribution = {
    "simple": 1.0,
    "reasoning": 0.0,
    "multi_context": 0.0,
    "conditional": 0.0,
}

# percentage of conversational question
chat_qa = 0

testset_generator = TestsetGenerator(
    generator_llm,
    critic_llm,
    embeddings_model,
    testset_distribution=testset_distribution,
    chat_qa=chat_qa,
)

evalset = testset_generator.generate(pages, test_size=2)
evalset_df = evalset.to_pandas()
evalset_df.head()

/Users/chan.k.20/.local/lib/python3.11/site-packages/ragas/testset/testset_generator.py:314: UserWarning: Most documents are too short
  warnings.warn("Most documents are too short")
 50%|█████     | 1/2 [00:42<00:42, 42.37s/it]/Users/chan.k.20/.local/lib/python3.11/site-packages/ragas/testset/testset_generator.py:264: UserWarning: No neighbors exists
  warnings.warn("No neighbors exists")
3it [01:05, 21.84s/it]                       


,question,context,answer,question_type,episode_done
0,Who is Batman's most enduring romance througho...,- While most of Batman's romantic relationship...,Catwoman is Batman's most enduring romance thr...,simple,True
1,Who created the character Batman and when did ...,The character was created by artist Bob Kane a...,The character Batman was created by artist Bob...,simple,True


In [4]:
# testsetgenerator = TestsetGenerator.from_default()
# test_size = 2
# testset = testsetgenerator.generate(pages, test_size=test_size)
# test_df = testset.to_pandas()
# test_df.head()

In [5]:
evalset_df['question_type'].value_counts()

question_type
simple    2
Name: count, dtype: int64

# Save test set

In [6]:
# test_df.to_csv("../data/batman_eval_simple.csv", index=False)